# Brute force of the simplified task

1. There are orders with the given duration.
2. There are 3 slots for matricies.
3. Define time steps - three or less orders start at the same time. Finish time is max of the three ones!!!


The simplest way:

1. Sort by duration.
2. Group be 3.
3. Get the final combination

In [1]:
import pandas as pd

from pulp import combination
from itertools import product
from collections import Iterable

In [32]:
input_file = "../data/orders.xlsx"
output_file = "../result/result_slot.xlsx"

In [33]:
sheetname = "Orders"
df_orders = pd.read_excel(input_file, sheetname, nrows=None)
df_orders

,MatrixName,OrderSize
0,Орион - 600,734
1,Орион - 700,722
2,Орион - 800,1622
3,Весна - 600,218
4,Весна - 700,350
5,Весна - 800,820
6,Сафари,292
7,Сафари 2,352
8,Сатурн ПГ,42
9,Сатурн ПО,584


NORDERS -  number of orders

NSLOTS - number of press matricies

NSTEPS3 - number of time steps when all 3 slots are used

NREST - the last time step, where 0 or 1 or 2 slots are used

ORDERS - set of names of orders

DURATION  - dict of durations of the given order

In [4]:
NORDERS = df_orders.shape[0]
NSLOTS = 3
NSTEPS_ALL_SLOTS, NREST = divmod(NORDERS, NSLOTS)

In [5]:
ORDERS = df_orders.index.values
ORDERS

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12], dtype=int64)

In [6]:
def flatten(lst):
    """Make flat nested lists or tuples."""
    for item in lst:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:        
            yield item

In [7]:
DURATION = df_orders["OrderSize"].to_dict()
DURATION

{0: 734,
 1: 722,
 2: 1622,
 3: 218,
 4: 350,
 5: 820,
 6: 292,
 7: 352,
 8: 42,
 9: 584,
 10: 374,
 11: 30,
 12: 38}

## A. stupied long way

In [8]:
# combinations when all slots are full 
# time of the task is max duration of single orders
comb_all_slots = {}
for k in combination(ORDERS, NSLOTS):
    comb_all_slots[k] = max(DURATION[i] for i in k)
len(comb_all_slots)

286

In [9]:
# combinations of rests 
comb_rest = {}
for k in combination(ORDERS, NREST):
    comb_rest[k] = max(DURATION[i] for i in k)

In [10]:
#comb_rest

In [11]:
# different sequencies for time steps
# reject non unique on the fly
steps_sequence = {}
min_duration = 1e6
for seq in combination(list(comb_all_slots.keys()), NSTEPS_ALL_SLOTS):
    f1 = list(flatten(seq))
    if len(f1) != len(set(f1)):
        continue
   
    if len(comb_rest) == 0:
        rest_duration = 0
        r = None

    for r in comb_rest:
        f2 = f1 + list(r)
        if len(f2) == len(set(f2)):
            rest_duration = comb_rest[r]
            break
    #print(f2)
    #print(seq, r)
    # single sequence duration
    k = [i for i in seq]
    seq_duration = sum(comb_all_slots[k] for k in seq) + rest_duration
    if r:
        k.append(r)
        
    steps_sequence[tuple(k)] = seq_duration
    if seq_duration < min_duration:
        min_duration = seq_duration

len(steps_sequence), min_duration

(200200, 2944)

In [12]:
for k, v in steps_sequence.items():
    if v == min_duration:
        print(k)

((0, 2, 5), (1, 9, 10), (3, 8, 12), (4, 6, 7), (11,))


### Answer for A

MINDURATION = 2944

BESTSQQUENCE = ((0, 2, 5), (1, 9, 10), (3, 8, 12), (4, 6, 7), (11,))

Questions:

1. Why I got only one sequence?

## B. Smater fast way

In [34]:
df = df_orders.copy()

In [35]:
df = df.sort_values(by="OrderSize", ascending=False).reset_index()
df

,index,MatrixName,OrderSize
0,2,Орион - 800,1622
1,5,Весна - 800,820
2,0,Орион - 600,734
3,1,Орион - 700,722
4,9,Сатурн ПО,584
5,10,Соло,374
6,7,Сафари 2,352
7,4,Весна - 700,350
8,6,Сафари,292
9,3,Весна - 600,218


In [36]:
df["NumStep"] = 0
df.loc[:, "NumStep"] = df.index.values//NSLOTS
df

,index,MatrixName,OrderSize,NumStep
0,2,Орион - 800,1622,0
1,5,Весна - 800,820,0
2,0,Орион - 600,734,0
3,1,Орион - 700,722,1
4,9,Сатурн ПО,584,1
5,10,Соло,374,1
6,7,Сафари 2,352,2
7,4,Весна - 700,350,2
8,6,Сафари,292,2
9,3,Весна - 600,218,3


In [38]:
df_grp = df.groupby("NumStep")
df_grp

In [46]:
MINDURATION = df_grp["OrderSize"].max().sum()
MINDURATION

2944

In [56]:
BESTSEQUENCE = tuple(tuple(df.loc[k, "index"] for k in i.values) for i in df_grp.groups.values())
BESTSEQUENCE

((2, 5, 0), (1, 9, 10), (7, 4, 6), (3, 8, 12), (11,))

## Conclusions

1. MINDURATION values coincide for A. and B.
2. BESTSQUENCES of A and B also coincide.

```
A = ((0, 2, 5), (1, 9, 10), (3, 8, 12), (4, 6, 7), (11,))
B = ((2, 5, 0), (1, 9, 10), (7, 4, 6),  (3, 8, 12), (11,))
```

In [54]:
df.OrderSize.sum()/3

2059.3333333333335